# Bronze to Silver - Estabelecimento

Os dados de bronze agora passam por um tratamento de dados para ficarem padronizados e já estejam em uma estrutura confiável para ser utilizada.

Selecionando catalogo `saude_sus`

In [0]:
spark.sql('USE CATALOG saude_sus')

Imports e variavels globais:

In [0]:
import pyspark.sql.functions as sf
from pyspark.sql.types import *

MAX_INT = 2147483647

Funcoes para tratamento dos dados:

In [0]:
def apply_casting(df, max_int):
    for column_name in df.columns:
        
        if column_name.startswith('FLG_'):
            df = df.withColumn(
                column_name,
                sf.when(sf.col(column_name).isNull(), None)
                .when(sf.trim(sf.col(column_name)).isin('NAO', 'N', '0', 'no', 'false'), False)
                .when(sf.trim(sf.col(column_name)).isin('SIM', 'S', '1', 'yes', 'true'), True)
                .when(sf.expr(f"try_cast({column_name} as int)") == 0, False)
                .when(sf.expr(f"try_cast({column_name} as int)") == 1, True)
                .otherwise(None).cast(BooleanType())
            )

        elif column_name == 'NUM_LOGRADOURO_ESTABELECIMENTO':
            df = df.withColumn(column_name, sf.expr("try_cast(NUM_LOGRADOURO_ESTABELECIMENTO as long)"))
            
            df = df.withColumn(column_name, 
                sf.when(
                    (sf.col(column_name) > max_int) | 
                    (sf.col(column_name) <= 0),
                    None
                ).otherwise(sf.col(column_name))
            )

        elif column_name.startswith(("COD_", "NUM_")):
            df = df.withColumn(column_name, sf.expr(f"try_cast({column_name} as int)"))

        elif column_name.startswith("DAT_"):
            df = df.withColumn(column_name, sf.col(column_name).cast(DateType()))

        elif "LATITUDE" in column_name or "LONGITUDE" in column_name:
            df = df.withColumn(column_name, sf.expr(f"try_cast({column_name} as double)"))

        else:
            df = df.withColumn(column_name, sf.col(column_name).cast(StringType()))
            
    return df

def normalize_phone_number(df):
    for column_name in df.columns:
        if column_name.startswith('TEL_'):
            df = df.withColumn(column_name, sf.regexp_replace(sf.col(column_name), '[^0-9]', ''))
    return df

In [0]:
df_bronze = spark.read.table('saude_sus.raw.raw_estabelecimento')

print(f'Schema detectado: {df_bronze.printSchema()}')

In [0]:
df_bronze = df_bronze.drop("_airbyte_generation_id", "_airbyte_meta", "_airbyte_raw_id", "_airbyte_extracted_at")

In [0]:
new_columns = {
    'codigo_uf' : 'COD_UF',
    'codigo_cnes' : 'COD_CNES',
    'numero_cnpj' : 'DSC_DOCUMENTO',
    'tipo_gestao' : 'DSC_TIPO_GESTAO',
    'nome_fantasia' : 'NOM_FANTASIA',
    'codigo_municipio' : 'COD_MUNICIPIO',
    'data_atualizacao' : 'DAT_ATUALIZACAO',
    'nome_razao_social' : 'NOM_RAZAO_SOCIAL',
    'codigo_tipo_unidade' : 'COD_TIPO_UNIDADE',
    'bairro_estabelecimento': 'END_BAIRRO_ESTABELECIMENTO',
    'numero_estabelecimento': 'NUM_LOGRADOURO_ESTABELECIMENTO',
    'endereco_estabelecimento': 'END_LOGRADOURO_ESTABELECIMENTO',
    'codigo_cep_estabelecimento': 'END_CEP_ESTABELECIMENTO',
    'descricao_turno_atendimento': 'DSC_TURNO_ATENDIMENTO',
    'codigo_estabelecimento_saude': 'DSC_ESTABELECIMENTO_SAUDE',
    'endereco_email_estabelecimento': 'DSC_EMAIL_ESTABELECIMENTO',
    'codigo_atividade_ensino_unidade': 'COD_ATIVIDADE_ENSINO',
    'descricao_esfera_administrativa': 'DSC_ESFERA_ADMINISTRATIVA',
    'numero_telefone_estabelecimento': 'TEL_ESTABELECIMENTO',
    'codigo_esfera_administrativa_unidade': 'COD_ESFERA_ADMINISTRATIVA',
    'estabelecimento_possui_servico_apoio': 'FLG_POSSUI_SERVICO_APOIO',
    'latitude_estabelecimento_decimo_grau': 'END_LATITUDE_ESTABELECIMENTO',
    'longitude_estabelecimento_decimo_grau': 'END_LONGITUDE_ESTABELECIMENTO',
    'codigo_identificador_turno_atendimento': 'COD_IDENTIFICADOR_TURNO_ATENDIMENTO',
    'estabelecimento_possui_centro_neonatal': 'FLG_POSSUI_CENTRO_NEONATAL',
    'estabelecimento_possui_centro_cirurgico': 'FLG_POSSUI_CENTRO_CIRURGICO',
    'estabelecimento_possui_centro_obstetrico': 'FLG_POSSUI_CENTRO_OBSTETRICO',
    'descricao_natureza_juridica_estabelecimento': 'DSC_NATUREZA_JURIDICA',
    'estabelecimento_possui_atendimento_hospitalar': 'FLG_POSSUI_ATENDIMENTO_HOSPITALAR',
    'estabelecimento_possui_atendimento_ambulatorial': 'FLG_POSSUI_ATENDIMENTO_AMBULATORIAL',
    'estabelecimento_faz_atendimento_ambulatorial_sus': 'FLG_POSSUI_ATENDIMENTO_AMBULATORIAL_SUS'
}

df_renamed = df_bronze.withColumnsRenamed(new_columns)
display(df_renamed)

In [0]:
df_silver = apply_casting(df_renamed, MAX_INT)
df_silver = normalize_phone_number(df_silver)

In [0]:
display(df_silver)